https://blog.csdn.net/weixin_44613063/article/details/90815082

### 导入相关库

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
import cv2

### 获取训练集和测试集

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

# 加载训练集
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transform,
                               )
# 加载测试集
test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transform,
                              )

### 数据装载和预览

In [4]:
# dataset 参数用于指定我们载入的数据集名称
# batch_size参数设置了每个包中的图片数据个数
# 在装载的过程会将数据随机打乱顺序并进打包 shuffle

# 装载训练集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)
# 装载测试集
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=64,
                                          shuffle=True)

In [6]:
# 实现单张图片可视化

images, labels = next(iter(train_loader))
img = torchvision.utils.make_grid(images)

img = img.numpy().transpose(1, 2, 0)
std = [0.5, 0.5, 0.5]
mean = [0.5, 0.5, 0.5]
img = img * std + mean

print(labels)
cv2.imshow('train_data', img)
key_pressed = cv2.waitKey(0)

if key_pressed == 27:     # 键盘上Esc键的键值
    cv2.destroyAllWindows() 

tensor([1, 8, 6, 7, 9, 0, 8, 9, 4, 8, 9, 6, 8, 8, 2, 6, 6, 6, 3, 3, 0, 1, 0, 3,
        4, 2, 3, 5, 5, 9, 2, 0, 5, 5, 8, 6, 8, 4, 7, 5, 3, 1, 6, 7, 4, 5, 7, 0,
        4, 3, 4, 5, 6, 3, 9, 6, 2, 8, 7, 2, 0, 0, 1, 3])


### 搭建神经网络

In [7]:
# 卷积层使用 torch.nn.Conv2d
# 激活层使用 torch.nn.ReLU
# 池化层使用 torch.nn.MaxPool2d
# 全连接层使用 torch.nn.Linear


class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = torch.nn.Sequential(torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(stride=2,kernel_size=2))
        self.dense = torch.nn.Sequential(torch.nn.Linear(14*14*128,1024),
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p=0.5),
                                         torch.nn.Linear(1024,10))
    def forward(self,x):
        x = self.conv1(x)
        x = x.view(-1,14*14*128)
        x = self.dense(x)
        return x

In [8]:
model = Model()
if torch.cuda.is_available():
    model.cuda()#将所有的模型参数移动到GPU上
cost = torch.nn.CrossEntropyLoss()
optimzer = torch.optim.Adam(model.parameters())

In [9]:
print(model)

Model(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [14]:
#迭代次数
n_epochs = 5
 
for epoch in range(n_epochs):
    running_loss = 0.0
    running_correct = 0
    print("Epoch{}/{}".format(epoch,n_epochs))
    print("-"*10)
    
    for data in train_loader:
        #print("train ing")
        X_train,y_train = data
        #有GPU加下面这行，没有不用加
        X_train,y_train = X_train.cuda(),y_train.cuda()
        X_train,y_train = Variable(X_train),Variable(y_train)
        outputs = model(X_train)
        _,pred = torch.max(outputs.data,1)
        optimzer.zero_grad()
        loss = cost(outputs,y_train)
        
        loss.backward()
        optimzer.step()
        running_loss += loss.data
        running_correct += torch.sum(pred == y_train.data)
        
    testing_correct = 0
    for data in test_loader:
        X_test,y_test = data
        #有GPU加下面这行，没有不用加
        X_test,y_test = X_test.cuda(),y_test.cuda()
        X_test,y_test = Variable(X_test),Variable(y_test)
        outputs = model(X_test)
        _,pred = torch.max(outputs,1)
        testing_correct += torch.sum(pred == y_test.data)
        
    print("Loss is :{:.4f},Train Accuracy is:{:.4f}%,Test Accuracy is:{:.4f}".format(running_loss/len(train_dataset),
                                                                                     100*running_correct/len(train_dataset),
                                                                                     100*testing_correct/len(test_dataset)))

Epoch0/5
----------


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Loss is :0.0020,Train Accuracy is:96.0000%,Test Accuracy is:97.0000
Epoch1/5
----------
Loss is :0.0008,Train Accuracy is:98.0000%,Test Accuracy is:98.0000
Epoch2/5
----------
Loss is :0.0005,Train Accuracy is:98.0000%,Test Accuracy is:98.0000
Epoch3/5
----------
Loss is :0.0004,Train Accuracy is:99.0000%,Test Accuracy is:98.0000
Epoch4/5
----------
Loss is :0.0003,Train Accuracy is:99.0000%,Test Accuracy is:98.0000
